In [ ]:
using Revise

In [ ]:
import Pkg; Pkg.activate("../../../."); # Activate MyelinWaterOrientation project

In [ ]:
Revise.includet("../../../init.jl")
mxcall(:cd, 0, pwd()) # change MATLAB path to current path for saving outputs

In [ ]:
exteriorgrids, torigrids, interiorgrids, outercircles, innercircles, bdry = creategeometry(
    BlochTorreyParameters{Float64}(g_ratio = 2//3);
    N = 1, # Number of circles
    bdry = Rectangle((-1.0,-1.0), (1.0,1.0)),
    outercircles = [Circle(Vec{2}((0.0,0.0)), 0.75)],
    PLOT = false,
    RESOLUTION = 0.125 # 1.0
);

In [ ]:
btparams = BlochTorreyParameters{Float64}(
    theta = π/2,
    g_ratio = 0.7,
    D_Tissue = 0.01, #0.0, #1e-3, #1.0e-1, # [μm²/s]
    D_Sheath = 0.01, #0.0, #1e-3, #1.0e-1, # [μm²/s]
    D_Axon = 0.01, #0.0, #1e-3, #1.0e-1, # [μm²/s]
    K_perm = 0.1, #10.0, # [μm/s]
    B0 = -3.0, # [T]
    R2_Tissue = 1.0, # [Hz]
    R2_sp = 1.0, # [Hz]
    R2_lp = 1.0, # [Hz]
    );

In [7]:
myelinprob, myelinsubdomains, myelindomains = createdomains(btparams,
    exteriorgrids, torigrids, interiorgrids, outercircles, innercircles);

"Assembling..." = "Assembling..."
  1.425377 seconds (27.97 M allocations: 527.654 MiB, 7.51% gc time)
  0.050783 seconds (119 allocations: 13.961 MiB, 9.32% gc time)
  0.428813 seconds (37.70 k allocations: 61.349 MiB, 1.93% gc time)
  0.040932 seconds (43 allocations: 14.409 MiB)


In [ ]:
K, M = getstiffness(myelindomains[1]), getmassfact(myelindomains[1]);
A = ParabolicLinearMap(getdomain(myelindomains[1]));
D = DiffEqParabolicLinearMapWrapper(A);
x = rand(size(A,2));
y = similar(x);

In [ ]:
@btime $M\($K*$x);
@btime ($K)'*($M\$x)
@btime mul!($y, adjoint($A), $x);
@btime mul!($y, $A, $x);
@btime $M\($K*$x);
@btime mul!($y, adjoint($K), $M\$x);

In [ ]:
A*x ≈ M\(K*x) && A'*x ≈ K'*(M\x)

### Setting Paths

In [ ]:
using Revise

In [ ]:
import Pkg; Pkg.activate("../../../."); # Activate MyelinWaterOrientation project

# Steady-State Heat Equation

## Introduction

The heat equation is the "Hello, world!" equation of finite elements.
Here we solve the equation on a unit square, with a uniform internal source.
The strong form of the (linear) heat equation is given by

\begin{equation}
 -\nabla \cdot (k \nabla u) = f  \quad x \in \Omega,
\end{equation}

where $u$ is the unknown temperature field, $k$ the heat conductivity,
$f$ the heat source and $\Omega$ the domain. For simplicity we set $f = 1$
and $k = 1$. We will consider homogeneous Dirichlet boundary conditions such that
\begin{equation}
u(x) = 0 \quad x \in \partial \Omega,
\end{equation}
where $\partial \Omega$ denotes the boundary of $\Omega$.

The resulting weak form is given by
\begin{equation}
\int_{\Omega} \nabla v \cdot \nabla u \ d\Omega = \int_{\Omega} v \ d\Omega,
\end{equation}
where $v$ is a suitable test function.

## Commented Program

Now we solve the problem in JuAFEM. What follows is a program spliced with comments.

First we load the JuAFEM package.

In [ ]:
using JuAFEM, SparseArrays

We start  generating a simple grid with 20x20 quadrilateral elements
using `generate_grid`. The generator defaults to the unit square,
so we don't need to specify the corners of the domain.

In [ ]:
# grid = generate_grid(Quadrilateral, (20, 20));
# grid = generate_grid(Triangle, (20, 20));
# grid = generate_grid(Triangle, (100, 100));
# grid = generate_grid(Triangle, (2, 4));
# grid = generate_grid(Triangle, (4, 8));
# grid = generate_grid(Triangle, (16, 32));
grid = generate_grid(Triangle, (32, 64));

### Trial and test functions
A `CellValues` facilitates the process of evaluating values and gradients of
test and trial functions (among other things). Since the problem
is a scalar problem we will use a `CellScalarValues` object. To define
this we need to specify an interpolation space for the shape functions.
We use Lagrange functions (both for interpolating the function and the geometry)
based on the reference "cube". We also define a quadrature rule based on the
same reference cube. We combine the interpolation and the quadrature rule
to a `CellScalarValues` object.

In [ ]:
dim = 2
# ip = Lagrange{dim, RefCube, 1}()
# qr = QuadratureRule{dim, RefCube}(2)
ip = Lagrange{dim, RefTetrahedron, 1}()
qr = QuadratureRule{dim, RefTetrahedron}(3)
cellvalues = CellScalarValues(qr, ip);

### Degrees of freedom
Next we need to define a `DofHandler`, which will take care of numbering
and distribution of degrees of freedom for our approximated fields.
We create the `DofHandler` and then add a single field called `u`.
Lastly we `close!` the `DofHandler`, it is now that the dofs are distributed
for all the elements.

In [ ]:
dh = DofHandler(grid)
push!(dh, :u, 1)
close!(dh);

Now that we have distributed all our dofs we can create our tangent matrix,
using `create_sparsity_pattern`. This function returns a sparse matrix
with the correct elements stored.

In [ ]:
K = create_sparsity_pattern(dh);

We can inspect the pattern using the `spy` function from `UnicodePlots.jl`.
By default the stored values are set to $0$, so we first need to
fill the stored values, e.g. `K.nzval` with something meaningful.

In [ ]:
using UnicodePlots
fill!(K.nzval, 1.0)
spy(K; height = 15)

### Boundary conditions
In JuAFEM constraints like Dirichlet boundary conditions
are handled by a `ConstraintHandler`.

In [ ]:
ch = ConstraintHandler(dh);

Next we need to add constraints to `ch`. For this problem we define
homogeneous Dirichlet boundary conditions on the whole boundary, i.e.
the `union` of all the face sets on the boundary.

In [ ]:
∂Ω = union(getfaceset.(Ref(grid), ["left", "right", "top", "bottom"])...);

Now we are set up to define our constraint. We specify which field
the condition is for, and our combined face set `∂Ω`. The last
argument is a function which takes the spatial coordinate $x$ and
the current time $t$ and returns the prescribed value. In this case
it is trivial -- no matter what $x$ and $t$ we return $0$. When we have
specified our constraint we `add!` it to `ch`.

In [ ]:
dbc = Dirichlet(:u, ∂Ω, (x, t) -> 0)
add!(ch, dbc);

We also need to `close!` and `update!` our boundary conditions. When we call `close!`
the dofs which will be constrained by the boundary conditions are calculated and stored
in our `ch` object. Since the boundary conditions are, in this case,
independent of time we can `update!` them directly with e.g. $t = 0$.

In [ ]:
close!(ch)
update!(ch, 0.0);

### Assembling the linear system
Now we have all the pieces needed to assemble the linear system, $K u = f$.
We define a function, `doassemble` to do the assembly, which takes our `cellvalues`,
the sparse matrix and our DofHandler as input arguments. The function returns the
assembled stiffness matrix, and the force vector.

In [ ]:
function doassemble(cellvalues::CellScalarValues{dim}, K::SparseMatrixCSC, dh::DofHandler) where {dim}
    # We allocate the element stiffness matrix and element force vector
    # just once before looping over all the cells instead of allocating
    # them every time in the loop.
    n_basefuncs = getnbasefunctions(cellvalues)
    Ke = zeros(n_basefuncs, n_basefuncs)
    fe = zeros(n_basefuncs)
    # Next we define the global force vector `f` and use that and
    # the stiffness matrix `K` and create an assembler. The assembler
    # is just a thin wrapper around `f` and `K` and some extra storage
    # to make the assembling faster.
    f = zeros(ndofs(dh))
    assembler = start_assemble(K, f)
    # It is now time to loop over all the cells in our grid. We do this by iterating
    # over a `CellIterator`. The iterator caches some useful things for us, for example
    # the nodal coordinates for the cell, and the local degrees of freedom.
    @inbounds for cell in CellIterator(dh)
        # Always remember to reset the element stiffness matrix and
        # force vector since we reuse them for all elements.
        fill!(Ke, 0)
        fill!(fe, 0)
        # For each cell we also need to reinitialize the cached values in `cellvalues`.
        reinit!(cellvalues, cell)
        # It is now time to loop over all the quadrature points in the cell and
        # assemble the contribution to `Ke` and `fe`. The integration weight
        # can be queried from `cellvalues` by `getdetJdV`.
        for q_point in 1:getnquadpoints(cellvalues)
            dΩ = getdetJdV(cellvalues, q_point)
            # For each quadrature point we loop over all the (local) shape functions.
            # We need the value and gradient of the testfunction `v` and also the gradient
            # of the trial function `u`. We get all of these from `cellvalues`.
            for i in 1:n_basefuncs
                v  = shape_value(cellvalues, q_point, i)
                ∇v = shape_gradient(cellvalues, q_point, i)
                fe[i] += v * dΩ
                for j in 1:n_basefuncs
                    ∇u = shape_gradient(cellvalues, q_point, j)
                    Ke[i, j] += (∇v ⋅ ∇u) * dΩ
                end
            end
        end
        # The last step in the element loop is to assemble `Ke` and `fe`
        # into the global `K` and `f` with `assemble!`.
        assemble!(assembler, celldofs(cell), fe, Ke)
    end
    return K, f
end

### Solution of the system
The last step is to solve the system. First we call `doassemble`
to obtain the global stiffness matrix `K` and force vector `f`.

In [ ]:
K, f = doassemble(cellvalues, K, dh);

To account for the boundary conditions we use the `apply!` function.
This modifies elements in `K` and `f` respectively, such that
we can get the correct solution vector `u` by using `\`.

In [ ]:
# apply!(K, f, ch)
# u = K \ f;

### Exporting to VTK
To visualize the result we export the grid and our field `u`
to a VTK-file, which can be viewed in e.g. [ParaView](https://www.paraview.org/).

In [ ]:
# vtk_grid("heat_equation", dh) do vtk
#     vtk_point_data(vtk, dh, u)
# end

*This notebook was generated using [Literate.jl](https://github.com/fredrikekre/Literate.jl).*

# Bloch-Torrey Equation

In [ ]:
Revise.includet("../../../init.jl")
mxcall(:cd, 0, pwd()) # change MATLAB path to current path for saving outputs

### Exterior domain

In [ ]:
# Treat whole domain as one "exterior" region; outer/innercircles should be empty, as well as tori/interior grids
exteriorgrids, torigrids, interiorgrids = [deepcopy(grid)], typeof(grid)[], typeof(grid)[];
outercircles, innercircles = Circle{2,Float64}[], Circle{2,Float64}[];

### Interior domain

In [ ]:
# Treat whole domain as one "interior" region
emptygrid = Grid(eltype(grid.cells)[], eltype(grid.nodes)[]);
exteriorgrids, torigrids, interiorgrids = [deepcopy(emptygrid)], [deepcopy(emptygrid)], [deepcopy(grid)];
outercircles, innercircles = [Circle(Vec{2}((0.0,0.0)), 2.0)], [Circle(Vec{2}((0.0,0.0)), 1.5)];

### Tori domain

In [ ]:
# Treat whole domain as one "tori" region
emptygrid = Grid(eltype(grid.cells)[], eltype(grid.nodes)[]);
exteriorgrids, torigrids, interiorgrids = [deepcopy(emptygrid)], [deepcopy(grid)], [deepcopy(emptygrid)];
outercircles, innercircles = [Circle(Vec{2}((0.0,-1.5)), 3.0)], [Circle(Vec{2}((0.0,-1.5)), 0.25)];

## Stiffness testing

In [ ]:
btparams = BlochTorreyParameters{Float64}(
    theta = π/2,
    D_Tissue = 1.0, #0.5, # [μm²/s]
    D_Sheath = 1.0, #0.5, # [μm²/s]
    D_Axon = 1.0, #0.5, # [μm²/s]
    K_perm = 0.0, #1.0 # [μm/s]
    B0 = 0.0, #-3.0 # [T]
    R2_Tissue = 0.0, # [Hz]
    R2_sp = 0.0, # [Hz]
    R2_lp = 0.0, # [Hz]
    );

In [ ]:
myelinprob, myelinsubdomains, myelindomains = createdomains(btparams,
    exteriorgrids, torigrids, interiorgrids, outercircles, innercircles);

In [ ]:
#NOTE: stiffness matrices are made with the opposite sign in `myelindomains`
K1 = -getstiffness(myelindomains[1])[1:2:end, 1:2:end];
K2 = -getstiffness(myelindomains[1])[2:2:end, 2:2:end];
M1 = getmass(myelindomains[1])[1:2:end, 1:2:end];
M2 = getmass(myelindomains[1])[2:2:end, 2:2:end];

In [ ]:
@assert M1 ≈ M2
@assert K1 ≈ K2
@assert K ≈ K1
@assert K ≈ K2

## $\omega$ contributions

In [ ]:
btparams = BlochTorreyParameters{Float64}(
    theta = π/2,
    D_Tissue = 0.0, # [μm²/s]
    D_Sheath = 0.0, # [μm²/s]
    D_Axon = 0.0, # [μm²/s]
    K_perm = 0.0, # [μm/s]
    B0 = -3.0, # [T]
    R2_Tissue = 0.0, # [Hz]
    R2_sp = 0.0, # [Hz]
    R2_lp = 0.0, # [Hz]
    );

In [ ]:
myelinprob, myelinsubdomains, myelindomains = createdomains(btparams,
    exteriorgrids, torigrids, interiorgrids, outercircles, innercircles);

In [ ]:
#NOTE: stiffness matrices are made with the opposite sign in `myelindomains`
K1 = -getstiffness(myelindomains[1])[1:2:end, 1:2:end];
K2 = -getstiffness(myelindomains[1])[2:2:end, 2:2:end];

In [ ]:
@assert maximum(abs, K1) ≈ 0 # Diagonals should be zero for omega testing
@assert maximum(abs, K2) ≈ 0

In [ ]:
#NOTE: stiffness matrices are made with the opposite sign in `myelindomains`
W = -getstiffness(myelindomains[1]);

In [ ]:
@assert W ≈ -W'

## $R_2$ contributions

In [ ]:
btparams = BlochTorreyParameters{Float64}(
    theta = π/2,
    D_Tissue = 0.0, # [μm²/s]
    D_Sheath = 0.0, # [μm²/s]
    D_Axon = 0.0, # [μm²/s]
    K_perm = 0.0, # [μm/s]
    B0 = 0.0, # [T]
    R2_Tissue = 1.0, # [Hz]
    R2_sp = 1.0, # [Hz]
    R2_lp = 1.0, # [Hz]
    );

In [ ]:
myelinprob, myelinsubdomains, myelindomains = createdomains(btparams,
    exteriorgrids, torigrids, interiorgrids, outercircles, innercircles);

In [ ]:
#NOTE: stiffness matrices are made with the opposite sign in `myelindomains`
R1 = -getstiffness(myelindomains[1])[1:2:end, 1:2:end];
R2 = -getstiffness(myelindomains[1])[2:2:end, 2:2:end];
M1 = getmass(myelindomains[1])[1:2:end, 1:2:end];
M2 = getmass(myelindomains[1])[2:2:end, 2:2:end];

In [ ]:
@assert R1 ≈ R2
@assert M1 ≈ M2
@assert R1 ≈ M1
@assert isposdef(M1) && isposdef(M2)

# Bloch-Torrey Propagation testing

### Zero-axon grids

In [ ]:
bdry = Rectangle((-1.0,-1.0), (1.0,1.0))
p, t = distmesh2d(
    x -> drectangle0(x, bdry), # fd
    x -> one(eltype(x)), # fh
    0.2, # h0
    [xmin(bdry) ymin(bdry); xmax(bdry) ymax(bdry)]; # bbox
    PLOT = false,
    PLOTLAST = false,
    MAXSTALLITERS = 10
);
exteriorgrids = [Grid(p,t)];
torigrids, interiorgrids = typeof(exteriorgrids[1])[], typeof(exteriorgrids[1])[];
outercircles, innercircles = Circle{2,Float64}[], Circle{2,Float64}[];

In [ ]:
# let g=interiorgrids[1], n=rand(1:getnnodes(g))
#     simpplot(g; newfigure = true);
#     mxcall(:hold, 0, "on");

#     nodes = [c.nodes for c in g.cells if n ∈ c.nodes]
#     nodes = unique(reduce(vcat, [nds...] for nds in nodes))
#     coords = g.nodes[nodes] .|> getcoordinates
    
#     X = reshape(reinterpret(Float64, coords), (2,length(coords)))' |> copy
#     mxcall(:scatter3, 0, X[:,1], X[:,2], ones(size(X,1),1), 100, "ro", "fill", "r")
#     mxcall(:scatter3, 0, Tuple(g.nodes[n].x)..., 1.0, 100, "bo", "fill", "b")
# end

### Single-axon grids

In [ ]:
exteriorgrids, torigrids, interiorgrids, outercircles, innercircles, bdry = creategeometry(
    BlochTorreyParameters{Float64}(g_ratio = 2//3);
    N = 1, # Number of circles
    bdry = Rectangle((-1.0,-1.0), (1.0,1.0)),
    outercircles = [Circle(Vec{2}((0.0,0.0)), 0.75)],
    PLOT = true,
    RESOLUTION = 0.5 # 1.0
);

In [ ]:
# # Keep only "exterior" region; outer/innercircles should be empty, as well as tori/interior grids
# exteriorgrids, torigrids, interiorgrids = exteriorgrids, typeof(grid)[], typeof(grid)[];
# outercircles, innercircles = Circle{2,Float64}[], Circle{2,Float64}[];

# # Keep only "tori" region
# emptygrid = Grid(eltype(grid.cells)[], eltype(grid.nodes)[]);
# exteriorgrids, torigrids, interiorgrids = [deepcopy(emptygrid)], torigrids, [deepcopy(emptygrid)];

# # Treat whole domain as one "interior" region
# emptygrid = Grid(eltype(grid.cells)[], eltype(grid.nodes)[]);
# exteriorgrids, torigrids, interiorgrids = [deepcopy(emptygrid)], [deepcopy(emptygrid)], interiorgrids;
# # exteriorgrids, torigrids, interiorgrids = [deepcopy(emptygrid)], [deepcopy(emptygrid)], [deepcopy(grid)];

# # Treat whole domain as adjacent "interior/tori" regions
# emptygrid = Grid(eltype(grid.cells)[], eltype(grid.nodes)[]);
# leftgrid = transform!(deepcopy(grid), x->Vec{2}(((1+x[1])/2 - 1, x[2])))
# rightgrid = transform!(deepcopy(grid), x->Vec{2}(((1+x[1])/2, x[2])))
# # exteriorgrids, torigrids, interiorgrids = [deepcopy(emptygrid)], [leftgrid], [rightgrid];
# exteriorgrids, torigrids, interiorgrids = [leftgrid], [rightgrid], [deepcopy(emptygrid)];

### Myelin domains

In [ ]:
btparams = BlochTorreyParameters{Float64}(
    theta = π/2,
    g_ratio = 0.7,
    D_Tissue = 0.01, #0.0, #1e-3, #1.0e-1, # [μm²/s]
    D_Sheath = 0.01, #0.0, #1e-3, #1.0e-1, # [μm²/s]
    D_Axon = 0.01, #0.0, #1e-3, #1.0e-1, # [μm²/s]
    K_perm = 0.1, #10.0, # [μm/s]
    B0 = 0.0, # [T]
    R2_Tissue = 1.0, # [Hz]
    R2_sp = 1.0, # [Hz]
    R2_lp = 1.0, # [Hz]
    );

In [ ]:
myelinprob, myelinsubdomains, myelindomains = createdomains(btparams,
    exteriorgrids, torigrids, interiorgrids, outercircles, innercircles);

### Propagate

In [23]:
# sols = solveblochtorrey(myelinprob, myelindomains;#, get_algfun(Tsit5());
#     tspan = (0.0, 1.0), # (0.0, 320.0e-3)
#     TE = 0.1, # 10e-3
#     ts = 0.0:0.1:1.0, # tspan[1]:TE/2:tspan[2], # tstops, which includes π-pulse times
#     u0 = Vec{2}((0.0, 1.0)), # initial π/2 pulse
#     cb = nothing, # MultiSpinEchoCallback(tspan; TE = TE),
#     reltol = 1e-4,
#     abstol = 1e-4
# );
sols = solveblochtorrey(myelinprob, myelindomains;#, get_algfun(Tsit5());
    tspan = (0.0, 320.0e-3),
    TE = 10e-3,
    reltol = 1e-4,
    abstol = 1e-4
);

i = 1/1: 
π-pulse at t = 5.0 ms
π-pulse at t = 15.0 ms
π-pulse at t = 25.0 ms
π-pulse at t = 35.0 ms
π-pulse at t = 45.0 ms
π-pulse at t = 55.0 ms
π-pulse at t = 65.0 ms
π-pulse at t = 75.0 ms
π-pulse at t = 85.0 ms
π-pulse at t = 95.0 ms
π-pulse at t = 105.0 ms
π-pulse at t = 115.0 ms
π-pulse at t = 125.0 ms
π-pulse at t = 135.0 ms
π-pulse at t = 145.0 ms
π-pulse at t = 155.0 ms
π-pulse at t = 165.0 ms
π-pulse at t = 175.0 ms
π-pulse at t = 185.0 ms
π-pulse at t = 195.0 ms
π-pulse at t = 205.0 ms
π-pulse at t = 215.0 ms
π-pulse at t = 225.0 ms
π-pulse at t = 235.0 ms
π-pulse at t = 245.0 ms
π-pulse at t = 255.0 ms
π-pulse at t = 265.0 ms
π-pulse at t = 275.0 ms
π-pulse at t = 285.0 ms
π-pulse at t = 295.0 ms
π-pulse at t = 305.0 ms
π-pulse at t = 315.0 ms
  3.535283 seconds (36.58 k allocations: 774.538 MiB, 4.41% gc time)
  3.699048 seconds (37.33 k allocations: 795.044 MiB, 5.18% gc time)


In [24]:
paramstr = "theta = $(rad2deg(btparams.theta)) deg, D = $(btparams.D_Tissue) um2/s, K = $(btparams.K_perm) um/s"
plotmagnitude(sols, btparams, myelindomains, bdry; titlestr = "Magnitude: " * paramstr)
# plotphase(sols, btparams, myelindomains, bdry; titlestr = "Phase: " * paramstr)

In [ ]:
simpplot(getgrid.(myelindomains); newfigure = true, axis = mxaxis(bdry), facecol = calcomega(myelinprob, myelinsubdomains))

In [ ]:
# let m = myelinsubdomains[1], d = getdomain(m), g=getgrid(d)#, n=rand(1:getnnodes(g))
#     dh = getdofhandler(d)
#     perm = zeros(Int, ndofs(dh))
#     for cell in CellIterator(getdofhandler(d))
#         for (i,n) in enumerate(cell.nodes)
#             perm[cell.celldofs[2i-1]] = 2n-1
#             perm[cell.celldofs[2i]] = 2n
#         end
#     end
#     @show perm
#     @show isperm(perm)
# end

In [ ]:
# let m = myelindomains[1], d = getdomain(m), g=getgrid(d)#, n=rand(1:getnnodes(g))
# #     dh = getdofhandler(d)
#     for n in 91#rand(1:getnnodes(g))
#         @show n
#         all_idxs = []
#         for cell in CellIterator(getdofhandler(d))
#             if n ∈ cell.nodes
# #                 @show cell.current_cellid[]
#                 @show cell.nodes
#                 @show cell.celldofs
#                 idx = findall(cell.nodes.==n)[]
#                 push!(all_idxs, cell.celldofs[2*idx-1:2*idx])
#             end
#         end
#         if length(unique(all_idxs)) != 1
#             @show n
#             @show all_idxs
#         end
#     end
# end

In [ ]:
# utest = map(myelinsubdomains) do m
#     Ktest = copy(getstiffness(m));
#     xtest = getcoordinates.(getnodes(getgrid(getdomain(m))));
#     circfuntest = x -> norm2(x - Vec{2}((0.5,0.5))) <= (0.5 + 1e-6)^2
#     ftest = reinterpret(Float64, [circfuntest(x) ? Vec{2}((0.0,1.0)) : zero(x) for x in xtest]) |> copy
#     utest = Ktest\ftest;
#     utest
# end

In [ ]:
# # simpplot(getgrid.(myelinsubdomains);
# #     newfigure = true, axis = mxaxis(bdry),
# #     facecol = norm.(reinterpret(Vec{2,Float64}, reduce(vcat, utest))))
# for (m,u) in zip(myelinsubdomains, utest)
#     simpplot(getgrid(m);
#         newfigure = true, axis = mxaxis(bdry),
#         facecol = norm.(reinterpret(Vec{2,Float64}, u)))
# end

In [ ]:
# mxcall(:scatter3, 0, [0.437048], [0.0315435], [1.0], ".")

In [ ]:
# getnnodes.(getgrid.(getdomain.(myelinsubdomains))) |> sum;
# getnnodes.(getgrid.(getdomain.([myelinsubdomains[1]])))

In [ ]:
# # spy(getstiffness(myedoms[1]); height = 30);
# # getstiffness(myedoms[1])[1,:] |> Array |> x -> x[.!(x.≈0)]
# Kmye = getstiffness(myedoms[1])[1:30,1:30] |> Array;
# length.([Kmye[i,.!(Kmye[i,:].≈0)] for i in 2:2:size(Kmye,1)]) |> sort

In [ ]:
# spy(getstiffness(adjdoms[1]); height = 30);
# getstiffness(adjdoms[1])[rand(1:180),:] |> Array |> x -> x[.!(x.≈0)]

In [ ]:
# adjdoms = deepcopy(myelindomains); adjsols = deepcopy(sols);
# myedoms = deepcopy(myelindomains); myesols = deepcopy(sols);

In [ ]:
calcsignal(adjsols, [1.0], adjdoms)

In [ ]:
calcsignal(myesols, [1.0], myedoms)

In [ ]:
for f in [maximum, minimum, median, mean]
    u1 = adjsols[1].u[end][2:2:end]
    u2 = myesols[1].u[end][2:2:end]
    @show f, f(u1), f(u2)
end

In [ ]:
histogram(adjsols[1].u[end][2:2:end]; nbins = 25)

In [ ]:
histogram(myesols[1].u[end][2:2:end]; nbins = 25)

In [ ]:
g1, g2 = deepcopy(interiorgrids[1]), deepcopy(getgrid(myelindomains[1]));

In [ ]:
[getfield(g1,f) == getfield(g2,f) for f in fieldnames(typeof(g1))]

In [ ]:
# getfaceset(getgrid(myelindomains[1]), "boundary")
# union(getfaceset.((exteriorgrids[1], interiorgrids[1], torigrids[1]), "boundary")...) |> length

In [ ]:
# getfaceset.(getgrid.(myelinsubdomains), "boundary") .|> length

In [ ]:
getstiffness(myelindomains[1])

In [ ]:
# @show exteriorgrids
# @show interiorgrids
# @show torigrids

In [ ]:
function disp_grid_nodes(grid)
    for c in grid.cells
        @show c.nodes
        println(collect(grid.nodes[n].x for n in c.nodes))
    end
end

In [ ]:
using DistMesh;
DistMesh.boundedges(interiorgrids[1])

In [ ]:
# _tmp = falses(3, getncells(interiorgrids[1]));
# ns = [];
# for (c,f) in interiorgrids[1].facesets["boundary"]
#     _tmp[f,c] = true
#     ce = interiorgrids[1].cells[c]
#     fe = JuAFEM.faces(ce)[f]
# #     @show JuAFEM.faces(ce)
#     n1 = interiorgrids[1].nodes[fe[1]].x
#     n2 = interiorgrids[1].nodes[fe[2]].x
# #     @show norm(n1), norm(n2)
# #     @show n1, n2
#     push!(ns, (n1,n2))
# end;
# _tmp';
# sort!(ns; by = x->angle(x[1]))

In [ ]:
K, M = -getstiffness(myelindomains[1]), getmass(myelindomains[1]);

In [ ]:
isposdef(K), isposdef(M)
# cond(Array(K)), cond(Array(M))

In [ ]:
exp(-1.0), exp(-0.1)

In [ ]:
(830, 1114, 476, 618)

In [ ]:
sols[1].u[end][294], sols[1].u[end][734]

In [ ]:
sols[1].u[end][830], sols[1].u[end][618]

In [ ]:
sols[1].u[end][616], sols[1].u[end][940]

In [ ]:
sols[1].u[end][1114], sols[1].u[end][476]